<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [11]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
nlp = spacy.load('en_core_web_lg')

In [12]:
# Function to tokenize text less stopwords
def tokenize(doc):
    tokenized = [token for token in simple_preprocess(doc) if token not in STOPWORDS]
    return tokenized

In [13]:
# Create tokenized column using the tokenize function
yelp['tokenized'] = yelp['text'].apply(lambda x: tokenize(x))
yelp['tokenized'].head()

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
Name: tokenized, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [57]:
# Instantiate vectorizer object
vect = TfidfVectorizer(stop_words = 'english')

# fit transform and convert values into a dataframe
sparse = vect.fit_transform(yelp['text'].values)
dtm = pd.DataFrame(sparse.todense(),columns=vect.get_feature_names())
dtm.head()

,00,000,001695,007,00a,00am,00ish,00pm,01,011802,...,誰も乗車しなくても,豆腐花,質問にも丁寧に答えてくれましたし,車好きさんには,這是一個不錯的選擇,運転しない,運転中も英語で指導があります,食べ物はうまい,餐後點了甜點,３時間後の便
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.neighbors import NearestNeighbors

In [58]:
# fit nearest neighbors model with the dataframe
nn  = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [40]:
# create fake review
fake_review = ["This restaurant is not good, food and service is bad"]

# transform fake review text
transformed_text = vect.transform(fake_review)

# apply model to the fake review text
_, indices = nn.kneighbors(transformed_text.todense())
print(transformed_text)

  (0, 24488)	0.22658871258089072
  (0, 21583)	0.2982444743360554
  (0, 20255)	0.4360727982935584
  (0, 16607)	0.2648178764773089
  (0, 12935)	0.41320445414093515
  (0, 10732)	0.28776736945578174
  (0, 9804)	0.2822753146109736
  (0, 2293)	0.49349846961714594
  (0, 1500)	0.15050489409590942


In [41]:
# print out similar reviews to the fake review created above
similar_topics = [yelp.text[t] for t in indices]
print(similar_topics, sep="\n\n")

[9776    The food here is not very good.  The service i...
4599    only 1 word describes this place............. ...
151     Great authentic chinese food. Service is not t...
6570    This is the best place for Mexican food late a...
2518    Don't waste your money. Bad management!!!  Bad...
5608    I have been to this place a few times, and it ...
8614    Worst food in Tempe and Phoenix. Every dish ha...
8134    Been eating here since they opened and I give ...
3129    This is a very nice restaurant. I was treated ...
9352    A good find. My cousin treated my wife and mys...
Name: text, dtype: object]


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [59]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [69]:
# instantiate vectorizer and classifier
vect1 = TfidfVectorizer()
sgdc = SGDClassifier()

# create pipeline
pipe = Pipeline([('vect', vect1),
                  ('sgdc', sgdc)])

# fit pipeline with data
pipe.fit(yelp['text'], yelp['stars'])

# predict number of stars of te fake review created
print('Predicted star review from fake review: ', pipe.predict(fake_review))

Predicted star review from fake review:  [1]


In [61]:
# fine tune model with grid search
parameters = {
    'vect__max_df': (0.5, 0.75),
    'vect__min_df': (.02, .05),
    'vect__max_features': ( 100, 500, 1000),
    'sgdc__max_iter': (300, 1000),
}
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=7)
grid_search.fit(yelp['text'], yelp['stars'])

C:\Users\showi\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=7,
       param_grid={'vect__max_df': (0.5, 0.75), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'sgdc__max_iter': (300, 1000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [67]:
# print out grid search score
print(f'Grid Search best score: {grid_search.best_score_}')
grid_search.predict(fake_review)

Grid Search best score: 0.6085


array([1], dtype=int64)

In [70]:
# Stretch Goal

vect2 = CountVectorizer()
sgdc2 = SGDClassifier()
pipe2 = Pipeline([('vect', vect2),
                  ('sgdc', sgdc2)])

parameters2 = {
    'vect__max_df': (0.5, 0.75),
    'vect__min_df': (.02, .05),
    'vect__max_features': ( 100, 500, 1000),
    'sgdc__max_iter': (300, 1000),
}
grid_search2 = GridSearchCV(pipe2, parameters2, cv=5, n_jobs=7)

grid_search2.fit(yelp['text'], yelp['stars'])

C:\Users\showi\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=7,
       param_grid={'vect__max_df': (0.5, 0.75), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'sgdc__max_iter': (300, 1000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [71]:
# compare scores of both vectorization methods
print(f'Best score with TF/IDF vectorizer is {grid_search.best_score_}')
print(f'Best score with Count vectorizer is {grid_search2.best_score_}')

Best score with TF/IDF vectorizer is 0.6085
Best score with Count vectorizer is 0.601


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [82]:
import gensim
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim import corpora
import os, sys, re

In [75]:
def review(df, tokens):
    """
    Generator function that takes a dataframe with a tokenized column and yields a tokenized 
    row for processing
    """
    assert tokens in df.columns, f"{tokens} does not exist!"
    
    for index, row in df.iterrows():
        token = row[tokens]
        yield token

Learn the vocubalary of the yelp data:

In [77]:
id2word = corpora.Dictionary(review(yelp, 'tokenized'))

Create a bag of words representation of the entire corpus

In [79]:
corpus = [id2word.doc2bow(rev) for rev in review(yelp, 'tokenized')]

Your LDA model should be ready for estimation: 

In [80]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [ ]:
topics = [' '.join(re.findall(r'"([^"]*)"',t[1])) for t in lda.print_topics()]
print(*topics, sep = "\n\n")

In [87]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\showi\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.000330  0.000047       1        1  10.493795
8     -0.003922  0.000121       2        1  10.461906
2     -0.000541  0.002995       3        1  10.302478
0      0.002843  0.002153       4        1  10.214583
9     -0.004645  0.000717       5        1  10.125676
3     -0.000955  0.001594       6        1  10.113271
4      0.003875 -0.000351       7        1  10.062673
1      0.002722  0.002108       8        1   9.797049
7     -0.000131 -0.005462       9        1   9.343365
5      0.001085 -0.003920      10        1   9.085204, topic_info=     Category         Freq        Term        Total  loglift  logprob
40    Default  4460.000000        good  4460.000000  30.0000  30.0000
208   Default  4133.000000       great  4133.000000  29.0000  29.0000
133   Default  4561.000000       place  4561.000000  28.0000  28.0000
144   Default  4755.000000        food  4755.000000  27.0000  27.0000
122   Default  1751.000000        best  1751.000000  26.0000  26.0000
18    Default  3330.000000        like  3330.000000  25.0000  25.0000
75    Default  1988.000000         got  1988.000000  24.0000  24.0000
34    Default  3304.000000        time  3304.000000  23.0000  23.0000
358   Default  1395.000000  definitely  1395.000000  22.0000  22.0000
37    Default  1441.000000        came  1441.000000  21.0000  21.0000
45    Default  3504.000000     service  3504.000000  20.0000  20.0000
92    Default  1688.000000        nice  1688.000000  19.0000  19.0000
381   Default  1373.000000      little  1373.000000  18.0000  18.0000
246   Default  1055.000000       right  1055.000000  17.0000  17.0000
165   Default   843.000000       think   843.000000  16.0000  16.0000
313   Default   995.000000        want   995.000000  15.0000  15.0000
735   Default  1589.000000        love  1589.000000  14.0000  14.0000
426   Default  1116.000000      better  1116.000000  13.0000  13.0000
41    Default   740.000000       lunch   740.000000  12.0000  12.0000
448   Default   831.000000         eat   831.000000  11.0000  11.0000
387   Default  1449.000000       order  1449.000000  10.0000  10.0000
33    Default   771.000000       small   771.000000   9.0000   9.0000
1143  Default   841.000000    location   841.000000   8.0000   8.0000
39    Default  1337.000000    friendly  1337.000000   7.0000   7.0000
351   Default  1403.000000        come  1403.000000   6.0000   6.0000
184   Default  1122.000000   recommend  1122.000000   5.0000   5.0000
131   Default  1382.000000      people  1382.000000   4.0000   4.0000
726   Default   643.000000        feel   643.000000   3.0000   3.0000
164   Default   598.000000       stars   598.000000   2.0000   2.0000
546   Default   645.000000     quality   645.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
376   Topic10    51.766216      inside   432.668030   0.2753  -6.7373
18    Topic10   301.193420        like  3330.165039  -0.0045  -4.9763
751   Topic10    53.849834     special   455.055023   0.2643  -6.6979
92    Topic10   162.076279        nice  1688.823364   0.0548  -5.5960
94    Topic10    98.939354       night   940.552795   0.1466  -6.0896
133   Topic10   362.728149       place  4561.662598  -0.1333  -4.7904
1266  Topic10   142.575317     ordered  1487.836914   0.0533  -5.7242
421   Topic10    85.966049       asked   808.716919   0.1570  -6.2301
180   Topic10   117.854416        menu  1193.352417   0.0835  -5.9146
131   Topic10   132.113007      people  1382.817749   0.0503  -5.8004
144   Topic10   356.261871        food  4755.639648  -0.1929  -4.8084
116   Topic10    90.931404       vegas   879.459229   0.1293  -6.1740
546   Topic10    69.935959     quality   645.967224   0.1754  -6.4365
526   Topic10    92.711388        told   930.528198   0.0923  -6.1546
208   Topic10   286.262817       great  4133.229004  -0.2714  -5.0272
371   Topic10    90

Based on the visualization above, people are generally satisfied with the place, using words such as "good, great, food, best, like." It appears that people are providing more positive reviews than negative reviews just from the eye test. For further analysis, we could remove these top positive words and dig further for more neutral or negative reviews to get an idea of what people may not like. Generally the topics are pretty spread out, with only a couple overlapping.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)